In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import pandas as pd

import cv2
from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor
from io import BytesIO
from urllib.request import urlopen
import librosa
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")


/home/mkoltunov/anaconda3/envs/eacl_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mkoltunov/anaconda3/envs/eacl_venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/mkoltunov/anaconda3/envs/eacl_venv/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/mkoltunov/anaconda3/envs/eacl_venv/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transfor

In [2]:
meld_path = '../../data/MELD.Raw'
train_data = pd.read_csv(os.path.join(meld_path, 'test_sent_emo.csv'))
train_data.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378"
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385"
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520"
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146"
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833"


In [3]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
model = Qwen2AudioForConditionalGeneration.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", device_map="auto", load_in_4bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 5/5 [00:04<00:00,  1.21it/s]


In [4]:
audio_desc_train = []

for i, row in tqdm(train_data.iterrows()):
    conversation = [
        {"role": "system", "content": "You are a helpful assistant that describes the speaker's tone and intonation in details."},
        {"role": "user", "content": [
            {"type": "audio", "audio_url": "no url"},
            {"type": "text", "text": "What speaker's voice sounds like?"}
        ]}
    ]

    text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
    
    # try: # ../data/MELD.Raw/train_splits/dia125_utt3.mp4
    try:
        video_path = os.path.join(meld_path, "output_repeated_splits_test", f"dia{row['Dialogue_ID']}_utt{row['Utterance_ID']}.mp4")
        au = librosa.load(video_path, sr=processor.feature_extractor.sampling_rate)[0]
    except:
        video_path = os.path.join(meld_path, "output_repeated_splits_test", f"final_videos_testdia{row['Dialogue_ID']}_utt{row['Utterance_ID']}.mp4")
        au = librosa.load(video_path, sr=processor.feature_extractor.sampling_rate)[0]
    audios = [au]
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True)
    # inputs.input_ids = inputs.input_ids.to("cuda")
    for key in inputs.keys():
        inputs[key] = inputs[key].to("cuda")

    generate_ids = model.generate(**inputs) # , max_length=512
    generate_ids = generate_ids[:, inputs.input_ids.size(1):]

    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    # except:
    #     response = "Nan"
    audio_desc_train.append(response)

0it [00:00, ?it/s]It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
1it [00:00,  1.14it/s]It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
2it [00:01,  1.64it/s]It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
3it [00:01,  1.98it/s]It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
4it [00:02,  2.25it/s]It is strongly

KeyboardInterrupt: 

In [9]:
train_data['audio_caption'] = audio_desc_train
train_data.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,audio_caption
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378",Neutral
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385",The speaker sounds happy.
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520",The speaker sounds like a woman.
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146",The speaker sounds angry.
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833","The speaker sounds male, aged between 16-25 ye..."


In [10]:
# train_data.to_csv("./data/MELD/modified_test_data.csv", index=False)